In [1]:
# https://github.com/insarlab/PyAPS/blob/244552cdfcf4e1a55de5f1439be4f08eb45872ec/src/pyaps3/processor.py#L182

'''Computes the delay function given Pressure, Temperature and Vapor pressure.

    Args:
        * Presi (np.ndarray) : Pressure at height levels.
        * Tempi (np.ndarray) : Temperature at height levels.
        * Vpri  (np.ndarray) : Vapor pressure at height levels.
        * hgt   (np.ndarray) : Height levels.
        * cdict (np.ndarray) : Dictionary of constants.

    Returns:
        * DDry2 (np.ndarray) : Dry component of atmospheric delay.
        * DWet2 (np.ndarray) : Wet component of atmospheric delay.

    .. note::
        Computes refractive index at each altitude and integrates the delay using cumtrapz.'''

'Computes the delay function given Pressure, Temperature and Vapor pressure.\n\n    Args:\n        * Presi (np.ndarray) : Pressure at height levels.\n        * Tempi (np.ndarray) : Temperature at height levels.\n        * Vpri  (np.ndarray) : Vapor pressure at height levels.\n        * hgt   (np.ndarray) : Height levels.\n        * cdict (np.ndarray) : Dictionary of constants.\n\n    Returns:\n        * DDry2 (np.ndarray) : Dry component of atmospheric delay.\n        * DWet2 (np.ndarray) : Wet component of atmospheric delay.\n\n    .. note::\n        Computes refractive index at each altitude and integrates the delay using cumtrapz.'

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxa
import matplotlib.pyplot as plt

from era import initconst

In [ ]:
era = xr.open_dataset('/bsuhome/zacharykeskinen/scratch/era5/ERA5_N45_N46_W116_W115_2020-01-01 00:00:00_0000.nc').sel(time = '2020-01-01')

era = era.transpose()
Presi = era['z'].data # 
Tempi = era['t'].data
Vpri = era['q'].data
hgt = era['level'].data

nhgt = len(hgt)              #Number of height points
nlat = Presi.shape[0]        #Number of stations
nlon = Presi.shape[1]
WonT = Vpri/Tempi
WonT2 = WonT/Tempi

cdict = initconst()
k1 = cdict['k1']
Rd = cdict['Rd']
Rv = cdict['Rv']
k2 = cdict['k2']
k3 = cdict['k3']
g = cdict['g']

#Dry delay
DDry2 = np.zeros((nlat,nlon,nhgt))
DDry2[:,:,:] = k1*Rd*(Presi[:,:,:] - Presi[:,:,-1][:,:,np.newaxis])*1.0e-6/g

import scipy

#Wet delay
S1 = scipy.integrate.cumtrapz(WonT,x=hgt,axis=-1)
val = 2*S1[:,:,-1]-S1[:,:,-2]
val = val[:,:,None]
S1 = np.concatenate((S1,val),axis=-1)
# del WonT

S2 = scipy.integrate.cumtrapz(WonT2,x=hgt,axis=-1)
val = 2*S2[:,:,-1]-S2[:,:,-2]
val = val[:,:,None]
S2 = np.concatenate((S2,val),axis=-1)
DWet2 = -1.0e-6*((k2-k1*Rd/Rv)*S1+k3*S2)

for i in range(nlat):
    for j in range(nlon):
        DWet2[i,j,:]  = DWet2[i,j,:] - DWet2[i,j,-1]

era['dry_delay'] = xr.DataArray(DDry2, coords = [era.longitude, era.latitude, era.level], dims = ['longitude','latitude', 'level'])
era['wet_delay'] = xr.DataArray(DWet2, coords = [era.longitude, era.latitude, era.level], dims = ['longitude','latitude', 'level'])

In [ ]:
era['wet_delay'].isel(level = 10).plot()